In [42]:
import pandas as pd
import urllib.request as urlrequest
from bs4 import BeautifulSoup
from datetime import datetime, timedelta


In [43]:
df = pd.read_csv("uscities.csv")
information = ['city','state_id','lat','lng']
loc_df = df.head(10)[information]
airport = ['JFK','LAX','ORD','MIA','DFW','PHL','HOU','ATL','WAS','BOS']
loc_df['airport'] = airport

In [44]:
base  = "http://forecast.weather.gov/MapClick.php?"
lat  = loc_df['lat'].to_numpy()
lng  = loc_df['lng'].to_numpy()
url = []
for lati,lngi in zip(lat,lng):
    url.append(f'{base}lat={lati}&lon={lngi}')

temp_day1 =[]
temp_day2 =[]
temp_day3 =[]


In [45]:
for i in range(10):
    page = urlrequest.urlopen(url[i]).read()
    soup = BeautifulSoup(page,'html.parser')
    weather = soup.find(id = 'seven-day-forecast-container')
    date = weather.find_all(class_='period-name')
    status = weather.find_all(class_='short-desc')
    temp = weather.find_all(class_='temp')
    day_count = 0
    for j in range(9):
        date_i=date[j].get_text()
        temp_i = temp[j].get_text()
        temp_digit = 0
        for word in temp_i.split():
            if word.isdigit():
                temp_digit = int(word)

        if(date_i != "Tonight" and date_i.find('Night') == -1):
            if(day_count == 0 ):
                temp_day1.append(temp_digit)
                day_count = day_count+1
            elif(day_count == 1):
                temp_day2.append(temp_digit)
                day_count = day_count+1
            elif(day_count == 2):
                temp_day3.append(temp_digit)
                day_count= day_count+1

In [46]:
loc_df['day1'] = temp_day1
loc_df['day2'] = temp_day2
loc_df['day3'] = temp_day3

In [47]:
today = datetime.today()

tomorrow = today + timedelta(days=1)
tomorrow_str = str(tomorrow.year) + "-"+ str(tomorrow.month) + "-"+ str(tomorrow.day)
day_after_today = today + timedelta(days=2)
day_after_today_str = str(day_after_today.year) + "-"+ str(day_after_today.month) + "-"+ str(day_after_today.day)
day_after_tomorrow = today + timedelta(days=3)
day_after_tomorrow_str = str(day_after_tomorrow.year) + "-"+ str(day_after_tomorrow.month) + "-"+ str(day_after_tomorrow.day)


In [48]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

depart = 'DTW'
#destinations = loc_df['airport'].tolist()[0:6] + loc_df['airport'].tolist()[8:10]
destinations = ['ORD' ]
dates = [tomorrow_str,day_after_today_str,day_after_tomorrow_str]

final_df = pd.DataFrame({'depart': [],
                         'arrive': [],
                         'date': [],
                         'depart_time': [],
                         'arrival_time': [],
                         'price': [],
                         'airline': [],
                         'duration': []})

for destination in destinations:

    time.sleep(20)

    for date in dates:

        url = f'https://www.kayak.com/flights/{depart}-{destination}/{date}?sort=bestflight_a'

        driver = webdriver.Chrome("/Users/haoyuhuang/Downloads/chromedriver")
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'lxml')

        # departure times
        departure_time = []
        departure_times = soup.findAll('span', attrs={'class': 'depart-time base-time'})
        for times in departure_times:
            departure_time.append(times.text)

        # arrival times
        arrival_time = []
        arrival_times = soup.findAll('span', attrs={'class': 'arrival-time base-time'})
        for times in arrival_times:
            arrival_time.append(times.text)

        # prices
        price = []
        price_tag = re.compile('Common-Booking-MultiBookProvider(.*) Theme-featured-large(.*) multi-row(.*)')
        prices = soup.findAll('div', attrs={'class': price_tag})
        for price_i in prices:
            price.append(price_i.text.split('\n')[4].strip())

        # airlines
        airline = []
        airlines = soup.findAll('div', attrs={'class': 'bottom', 'dir': 'ltr'})
        for airline_i in airlines:
            airline.append(airline_i.text.replace('\n', ''))

        # durations
        duration = []
        durations = soup.findAll('div', attrs={'class': 'section duration allow-multi-modal-icons'})
        for duration_i in durations:
            duration.append(' '.join(duration_i.text.split(' ')[:2]).replace('\n', ''))

        df = pd.DataFrame({'depart': depart,
                           'arrive': destination,
                           'date': date,
                           'depart_time': departure_time[:5],
                           'arrival_time': arrival_time[:5],
                           'price': price[:5],
                           'airline': airline[:5],
                           'duration': duration[:5]})

        final_df = pd.concat([final_df, df], ignore_index=True, sort=False)
        driver.close()
        time.sleep(20)

final_df.to_csv('flight.csv',index = False)

<ipython-input-48-38dc91eb6c43>:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/haoyuhuang/Downloads/chromedriver")


In [41]:
newdf = pd.merge(loc_df, final_df, left_on="airport", right_on="arrive_at")
newdf = newdf.drop(['state_id','lat','lng'],axis=1)
newdf.to_csv("final_result.csv")